In [0]:
import pandas as pd

PUB_WEIGHTS = {
  'Livro Publicado ou Organizado ': 9,
  'Artigo Publicado': 8,
  'Capítulo de Livro Publicado': 8,
  'Artigo Aceito para Publicação': 7,
  'Texto em Jornal/Revista': 6,
  'Trabalho em Evento': 3,
  'Outra Produção Bibliográfica': 2,
  'Prefácio/Posfácio': 2,
  'nan': 1
}

network_data = pd.read_csv('trust_network.csv')
network_data['pub_type'] = network_data['pub_type'].astype(str)
network_data['type_weitght'] = network_data.pub_type.apply(lambda p: PUB_WEIGHTS[p])

In [0]:
from itertools import combinations

def cluster(pubid):
    return network_data[network_data['pubid'] == pubid]

def authname(authid):
    return network_data[network_data['authid'] == authid]['author'].iloc[0]

def edges(pubid):
    pub_cluster = cluster(pubid)
    authors = pub_cluster['authid'].unique()
    weight = pub_cluster['type_weitght'].iloc[0]

    return [(*edge, {'weight': weight, 'pubid': pubid}) for edge in combinations(authors, 2)]

In [0]:
import networkx as nx

trust_network = nx.MultiGraph()

for authid in network_data['authid'].unique():
    trust_network.add_node(authid, name=authname(authid)) 

for pubid in network_data['pubid'].unique():
    trust_network.add_edges_from(edges(pubid))

In [0]:
import numpy as np

def path_length(path_weights, alpha):
    return sum([1 / weight ** alpha for weight in path_weights])

def jumps(l, n=2):
    return [l[i:i+n] for i in range(len(l))][:-1]

def sum_weights(n, j):
    return sum([n[j[0]][j[1]][k]['weight'] for k in n[j[0]][j[1]]])

def sum_of_weights(jumps, network):
    return [sum_weights(network, jump) for jump in jumps]

def deg_arr(deg):
    return np.array([v for _, v in deg])

def degree_centrality(network, alpha):
    k = deg_arr(network.degree()) ** (1 - alpha)
    s = deg_arr(network.degree(weight='weight')) ** alpha
    centralities = k * s
    return [{node: centralities[i]} for i, node in enumerate(network.nodes)]

In [0]:
alpha = 0.5

[(p, path_length(sum_of_weights(jumps(p), trust_network), alpha)) for p in nx.all_simple_paths(trust_network, 14, 40, cutoff=6)]

[([14, 25, 125, 39, 118, 89, 40], 1.8838118415212128),
 ([14, 25, 125, 39, 118, 89, 40], 1.8838118415212128),
 ([14, 25, 125, 39, 118, 89, 40], 1.8838118415212128),
 ([14, 25, 125, 39, 118, 89, 40], 1.8838118415212128),
 ([14, 25, 125, 39, 118, 89, 40], 1.8838118415212128),
 ([14, 25, 125, 39, 118, 89, 40], 1.8838118415212128),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25, 125, 39, 144, 89, 40], 1.4528987074039548),
 ([14, 25,

In [0]:
alpha = 0.5

centralities = degree_centrality(trust_network, alpha)

sorted(centralities, key=lambda centrality: centrality[list(centrality.keys())[0]], reverse=True)

[{1: 236.92825918408298},
 {31: 226.8633950199988},
 {108: 210.35683967962632},
 {162: 184.0217378463751},
 {0: 183.25665062965658},
 {36: 180.32747988035544},
 {53: 179.55500549970753},
 {106: 143.10835055998655},
 {111: 133.0413469565007},
 {129: 129.18204209564112},
 {86: 127.67928571228772},
 {50: 118.94536560959406},
 {25: 118.28778466096996},
 {160: 117.16228061966018},
 {44: 113.08403954581742},
 {124: 103.69185117452575},
 {62: 101.82337649086284},
 {125: 100.52362906302179},
 {82: 95.9166304662544},
 {39: 95.69743988216194},
 {118: 94.86832980505139},
 {76: 86.53323061113574},
 {18: 81.24038404635961},
 {136: 81.24038404635961},
 {149: 79.54244150137711},
 {144: 77.7946013551069},
 {97: 72.64984514780468},
 {42: 69.7997134664606},
 {70: 68.93475175845634},
 {21: 68.92024376045111},
 {128: 68.24954212300622},
 {75: 68.00735254367721},
 {104: 67.88225099390856},
 {9: 67.08203932499369},
 {164: 64.24951361683604},
 {89: 60.332412515993425},
 {114: 58.32666628567074},
 {12: 57.157

236.92825918408298